In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1312418/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_13_6_2,0.942958,0.787235,0.968956,0.890587,0.063129,0.254450,0.054163,0.160197,"Hidden Size=[35], regularizer=0.01, learning_r..."
1,model_13_5_3,0.942357,0.918526,0.884673,0.905537,0.063793,0.127624,0.238282,0.179699,"Hidden Size=[35], regularizer=0.01, learning_r..."
2,model_13_5_4,0.942233,0.918157,0.881344,0.903675,0.063931,0.128203,0.245160,0.183241,"Hidden Size=[35], regularizer=0.01, learning_r..."
3,model_13_6_1,0.940772,0.812182,0.994229,0.915546,0.065548,0.224615,0.010068,0.123652,"Hidden Size=[35], regularizer=0.01, learning_r..."
4,model_13_5_5,0.939867,0.927364,0.851686,0.892530,0.066549,0.113780,0.306437,0.204442,"Hidden Size=[35], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
395,model_8_9_11,0.873334,0.927447,0.721547,0.799421,0.140182,0.057669,0.430668,0.233198,"Hidden Size=[20], regularizer=0.1, learning_ra..."
396,model_8_9_10,0.873331,0.927508,0.721500,0.799414,0.140185,0.057620,0.430740,0.233206,"Hidden Size=[20], regularizer=0.1, learning_ra..."
397,model_8_9_5,0.873323,0.927774,0.721355,0.799419,0.140194,0.057408,0.430964,0.233199,"Hidden Size=[20], regularizer=0.1, learning_ra..."
398,model_8_9_9,0.873321,0.927286,0.721584,0.799386,0.140196,0.057797,0.430610,0.233238,"Hidden Size=[20], regularizer=0.1, learning_ra..."
